In [1]:
import pprint

import pandas as pd
import requests

pp = pprint.PrettyPrinter()

# Exploring Fiskeridirektoratet's public data API

The public API is documented here:

    https://api.fiskeridir.no/pub-aqua/api/swagger-ui/index.html

I assume there may also be private endpoints with additional data (?).

In [2]:
# Base URL for the API
base_url = "https://api.fiskeridir.no/pub-aqua/api/v1/"

## 1. List all sites

The easiest way to get an overview of all sites seem to be using the `dump/new-legacy-csv` endpoint.

In [3]:
# Endpoint for sites overview
endpoint = "dump/new-legacy-csv"

# Read CSV
url = base_url + endpoint
site_df = pd.read_csv(url, sep=";", skiprows=1)
print(len(site_df), "sites in database.")

# Filter to salmon and rainbow trout in seawater
site_df = site_df.query(
    "(LOK_PLASS in ('SJØ', 'HAV')) and (ART_KODE in (71401, 71101))"
)
print(len(site_df), "sites producing salmon and/or trout in seawater.")

display(site_df.head())

# Show results for first site
print("\nResults for first site:\n", site_df.iloc[0].T, "\n")

25634 sites in database.
14192 sites producing salmon and/or trout in seawater.


,TILL_NR,ORG.NR/PERS.NR,NAVN,ADRESSE,POSTNR,POSTSTED,TILDELINGSTIDSPUNKT,TIDSBEGRENSET,TILL_KOMNR,TILL_KOM,...,LOK_KOMNR,LOK_KOM,LOK_PLASS,VANNMILJØ,LOK_KAP,LOK_ENHET,UTGÅR_DATO,N_GEOWGS84,Ø_GEOWGS84,PROD_OMR
144,AA L 0001,951375772.0,HELLESUND FISKEOPPDRETT AS,"LANGHOLMSUND, KJEHOLMEN 1",4770,HØVÅG,09-08-1973,NaN,4215,LILLESAND,...,4215,LILLESAND,SJØ,SALT,585.0,TN,NaN,58.143983,8.26705,1.0
146,AA L 0001,951375772.0,HELLESUND FISKEOPPDRETT AS,"LANGHOLMSUND, KJEHOLMEN 1",4770,HØVÅG,09-08-1973,NaN,4215,LILLESAND,...,4215,LILLESAND,SJØ,SALT,585.0,TN,NaN,58.143983,8.26705,1.0
147,AA L 0001,951375772.0,HELLESUND FISKEOPPDRETT AS,"LANGHOLMSUND, KJEHOLMEN 1",4770,HØVÅG,09-08-1973,NaN,4215,LILLESAND,...,4215,LILLESAND,SJØ,SALT,910.0,TN,NaN,58.140267,8.25530,1.0
149,AA L 0001,951375772.0,HELLESUND FISKEOPPDRETT AS,"LANGHOLMSUND, KJEHOLMEN 1",4770,HØVÅG,09-08-1973,NaN,4215,LILLESAND,...,4215,LILLESAND,SJØ,SALT,910.0,TN,NaN,58.140267,8.25530,1.0
150,AA L 0001,951375772.0,HELLESUND FISKEOPPDRETT AS,"LANGHOLMSUND, KJEHOLMEN 1",4770,HØVÅG,09-08-1973,NaN,4215,LILLESAND,...,4225,LYNGDAL,SJØ,SALT,1560.0,TN,NaN,58.055833,7.06875,1.0



Results for first site:
 TILL_NR                                 AA L 0001
ORG.NR/PERS.NR                        951375772.0
NAVN                   HELLESUND FISKEOPPDRETT AS
ADRESSE                 LANGHOLMSUND, KJEHOLMEN 1
POSTNR                                       4770
POSTSTED                                    HØVÅG
TILDELINGSTIDSPUNKT                    09-08-1973
TIDSBEGRENSET                                 NaN
TILL_KOMNR                                   4215
TILL_KOM                                LILLESAND
FORMÅL                                KOMMERSIELL
PRODUKSJONSFORM                           MATFISK
ART                                          Laks
ART_KODE                                    71101
TILL_KAP                                    804.0
TILL_ENHET                                     TN
LOK_NR                                      12832
LOK_NAVN                                RISHOLMEN
LOK_KOMNR                                    4215
LOK_KOM                 

## 2. Get details for location

In [4]:
# Endpoint with details for first site
loc_nr = site_df.iloc[0]["LOK_NR"]
endpoint = f"sites/{loc_nr}"

url = base_url + endpoint
response = requests.get(url)
response.raise_for_status()
site_data = response.json()
pp.pprint(site_data)

{'capacity': 585.0,
 'capacityUnitType': 'TN',
 'connections': [{'active': True,
                  'key': 'AA-L-0004|12832',
                  'licenseId': 24,
                  'licenseNr': 'AA-L-0004',
                  'registeredTime': '2020-01-09T08:45:18Z',
                  'siteId': 2808,
                  'siteName': 'RISHOLMEN',
                  'siteNr': '12832',
                  'status': 'ACTIVE',
                  'statusValue': 'AKTIV',
                  'temporaryUntil': None,
                  'validFrom': '2020-01-09T08:45:18Z',
                  'validUntil': '9999-12-30T00:00:00Z'},
                 {'active': True,
                  'key': 'AA-L-0001|12832',
                  'licenseId': 23,
                  'licenseNr': 'AA-L-0001',
                  'registeredTime': '2020-01-09T08:45:17Z',
                  'siteId': 2808,
                  'siteName': 'RISHOLMEN',
                  'siteNr': '12832',
                  'status': 'ACTIVE',
                  '

## 3. Get licence information

It seems that sites have "connections" and the connections link to licensing information. For example, the site above has two connections: `'licenseNr': 'AA-L-0004'` and `'licenseNr': 'AA-L-0001'`

In [5]:
lic_nr = site_data['connections'][0]['licenseNr']
endpoint = f"licenses/{lic_nr}"

url = base_url + endpoint
response = requests.get(url)
response.raise_for_status()
lic_data = response.json()
pp.pprint(lic_data)

{'capacity': {'accumulated': 804.0,
              'current': 804.0,
              'type': 'MTK',
              'unit': 'TN'},
 'connections': [{'active': True,
                  'key': 'AA-L-0004|29957',
                  'licenseId': 24,
                  'licenseNr': 'AA-L-0004',
                  'registeredTime': '2020-01-09T08:45:18Z',
                  'siteId': 5355,
                  'siteName': 'NAKKESTAD',
                  'siteNr': '29957',
                  'status': 'ACTIVE',
                  'statusValue': 'AKTIV',
                  'temporaryUntil': None,
                  'validFrom': '2020-01-09T08:45:18Z',
                  'validUntil': '9999-12-30T00:00:00Z'},
                 {'active': True,
                  'key': 'AA-L-0004|14016',
                  'licenseId': 24,
                  'licenseNr': 'AA-L-0004',
                  'registeredTime': '2020-01-09T08:45:18Z',
                  'siteId': 3845,
                  'siteName': 'AUENES',
                  